# GNB - Gaussian Naive Bayes

In [3]:
import collections
import numpy as np
from math import sqrt, exp, pi
from sklearn.model_selection import train_test_split
from src import utils as ut

Esta função é responsável por conseguir as probabilidades anteriores dado a label y.

In [4]:
def preprob(y):
    ydic = collections.Counter(y)
    preproba = np.ones(2)
    for i in range(0, 2):
        preproba[i] = ydic[i]/y.shape[0]
    return preproba

Essa classe recupera a média e a variância

In [5]:
def getmeanvar(X, y):
    columns = X.shape[1]
    lines = X.shape[0]
    m = np.ones(2, columns)
    v = np.ones(2, columns)
    n0 = np.bincount(y)[np.nonzero(np.bincount(y))[0]][0]
    x0 = np.ones((n0, columns))
    x1 = np.ones((lines - n0, columns))
    k = 0
    for i in range(0, lines):
        if y[i] == 0:
            x0[k] = X[i]
            k = k+1
    
    for j in range(0, columns):
        m[0][j] = np.mean(x0.T[j])
        v[0][j] = np.var(x0.T[j])*(n0/n0-1)
        m[1][j] = np.mean(x1.T[j])
        v[1][j] = np.var(x1.T[j])*((lines-n0)/((lines - n0) -1))
    return m, v

In [6]:
def nextprob(m, v, x):
    c = m.shape[1]
    pfc = np.ones(2)
    for i in range(0, 2):
        prod = 1
        for j in range(0, c):
            prod = prod * (1/sqrt(2*pi*v[i][j])) * exp(-0.5 * pow((x[j] - m[i][j]), 2)/v[i][j])
        pfc[i] = prod
    return pfc

In [7]:
def GNB(X, y, x):
    m,v = getmeanvar(X, y)
    prod = nextprob(m, v, x)
    pre = preprob(y)
    pcf = np.ones(2)
    total = 0
    for i in range(0,2):
        total = total + (prod[i] * pre[i])
    for i in range(0, 2):
        pcf[i] = (prod[i] * pre[i])/total
    pred = int(pcf.argmax())
    return m, v, pre, prod, pcf, pred 


In [8]:
def execute():
    X, y = ut.readdata()
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#Holdout 80/20
    c = 0
    for i, j in zip(x_test, y_test):
        m, v, pre, prod, pcf, pred = GNB(x_train, y_train, i)
        if(pred != j):
            c+=1
    
    print("Erro: ", (c/x_test.shape[0]))

In [9]:
execute()    

TypeError: data type not understood